In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [20]:
train_test = utils.read_df_pkl('../input/base_term**')
# df_hist = utils.read_df_pkl('../input/hist_clean*')
# df_new = utils.read_df_pkl('../input/new_clean*')
df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00,  4.91it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1971.22 Mb (0.0% reduction)
Mem. usage decreased to 129.17 Mb (0.0% reduction)


In [21]:
df_hist = df_hist.set_index(key).join(train_test.set_index(key)).reset_index()
df_new = df_new.set_index(key).join(train_test.set_index(key)).reset_index()
for df in [df_hist, df_new]:
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
#     df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
#     df['yyyymmdd'] = df['purchase_date'].map(lambda x: str(x)[:10])

In [22]:
df_new_lag1 = df_new[df_new['month_lag']==1]
df_new_lag2 = df_new[df_new['month_lag']==2]

df = df_hist

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]
print(auth1.shape)
print(auth0.shape)

cat1_0 = False
cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]
#     del auth1, auth0
#     gc.collect()
    auth1_cat1_lag0  = auth1_cat1[auth1_cat1['month_lag']==0]
    auth1_cat0_lag0  = auth1_cat0[auth1_cat0['month_lag']==0]
    auth1_cat1_lag1  = auth1_cat1[auth1_cat1['month_lag']==-1]
    auth1_cat0_lag1  = auth1_cat0[auth1_cat0['month_lag']==-1]
    auth1_cat1_lag2  = auth1_cat1[auth1_cat1['month_lag']==-2]
    auth1_cat0_lag2  = auth1_cat0[auth1_cat0['month_lag']==-2]
    auth1_cat1_lag02 = auth1_cat1[auth1_cat1['month_lag']>=-2]
    auth1_cat0_lag02 = auth1_cat0[auth1_cat0['month_lag']>=-2]
    auth1_cat1_lag05 = auth1_cat1[auth1_cat1['month_lag']>=-5]
    auth1_cat0_lag05 = auth1_cat0[auth1_cat0['month_lag']>=-5]
    
    new_cat1_lag1  = new_cat1[new_cat1['month_lag']==1]
    new_cat0_lag1  = new_cat0[new_cat0['month_lag']==1]
    new_cat1_lag2  = new_cat1[new_cat1['month_lag']==2]
    new_cat0_lag2  = new_cat0[new_cat0['month_lag']==2]

else:
    
    auth1_lag0 = auth1[auth1['month_lag']==0]
    auth1_lag1 = auth1[auth1['month_lag']==-1]
    auth1_lag2 = auth1[auth1['month_lag']==-2]
    auth1_lag02 = auth1[auth1['month_lag']>=-2]
    auth1_lag05 = auth1[auth1['month_lag']>=-5]
    
    auth0_lag0 = auth0[auth0['month_lag']==0]
    auth0_lag1 = auth0[auth0['month_lag']==-1]
    auth0_lag2 = auth0[auth0['month_lag']==-2]
    auth0_lag02 = auth0[auth0['month_lag']>=-2]
    auth0_lag05 = auth0[auth0['month_lag']>=-5]

(26595452, 29)
(2516909, 29)


In [24]:
prefix = '214_pst'
prefix = '314_pst'
prefix = '414_pst'
new_df_list = [new_cat1, new_cat0, new_cat1_lag1, new_cat0_lag1, new_cat1_lag2, new_cat0_lag2]
new_fname_list = ['new_cat1', 'new_cat0', 'new_cat1_lag1', 'new_cat0_lag1', 'new_cat1_lag2', 'new_cat0_lag2']

debug = False
if debug:
    train_test = train_test.head(10000)
        

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


#========================================================================
# 集計が必要なFeature
#========================================================================
for df, fname in zip(tqdm(new_df_list), new_fname_list):
    if debug:
        df = df.head(3000)
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # new
    print('Make Feature.')
   
    if ('level_0' and 'index' not in df.columns) and key not in df.columns:
        df.reset_index(inplace=True)
    
    aggs = {}
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['duration'] = df['purchase_amount']*df['month_diff']
    
    aggs['duration'] = ['sum', 'mean', 'std', 'max', 'skew']
    aggs['purchase_amount'] = ['sum']
    aggs['card_id'] = ['count']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    train_test = train_test.join(df_agg)
    print(train_test.shape)
    
    if not(fname.count('lag')):
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 2
    else:
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 1

    #========================================================================
    # Save Feature
    elo_save_feature(prefix=prefix, df_feat=train_test)
    #========================================================================
print('Complete!')

100%|██████████| 3/3 [00:00<00:00, 171.13it/s]

Make Feature.
(325540, 12)


100%|██████████| 3/3 [00:00<00:00, 175.49it/s]

Make Feature.
(325540, 12)


100%|██████████| 3/3 [00:00<00:00, 161.63it/s]

...

Make Feature.
(325540, 12)


100%|██████████| 3/3 [00:00<00:00, 179.88it/s]

Make Feature.
(325540, 12)


100%|██████████| 6/6 [02:00<00:00, 19.81s/it]

Complete!


In [25]:
debug = False
prefix = '214_pst'
prefix = '314_pst'
prefix = '414_pst'
hist_df_list = [
auth1_cat1 
,auth1_cat0 
,auth1_cat1_lag0 
,auth1_cat0_lag0 
,auth1_cat1_lag1 
,auth1_cat0_lag1 
,auth1_cat1_lag2 
,auth1_cat0_lag2 
,auth1_cat1_lag02
,auth1_cat0_lag02
,auth1_cat1_lag05
,auth1_cat0_lag05
]


hist_fname_list = [
'auth1_cat1'
,'auth1_cat0'
,'auth1_cat1_lag0'
,'auth1_cat0_lag0'
,'auth1_cat1_lag1'
,'auth1_cat0_lag1'
,'auth1_cat1_lag2'
,'auth1_cat0_lag2'
,'auth1_cat1_lag02'
,'auth1_cat0_lag02'
,'auth1_cat1_lag05'
,'auth1_cat0_lag05'
]

for df, fname in zip(tqdm(hist_df_list), hist_fname_list):
    if debug:
        df = df.head(3000)
    
#     df_train = utils.read_df_pkl('../input/train0*')
#     df_test = utils.read_df_pkl('../input/test0*')
#     df_train.set_index(key, inplace=True)
#     df_test.set_index(key, inplace=True)
#     train_test = pd.concat([df_train, df_test], axis=0)
    train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)[[target, 'hist_regist_term']]
    
#     hist
    print('Make Feature.')
    
    if ('level_0' and 'index' not in df.columns) and key not in df.columns:
        df.reset_index(inplace=True)

    aggs = {}
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['duration'] = df['purchase_amount']*df['month_diff']
    aggs['duration'] = ['sum', 'mean', 'std', 'max', 'skew']
    
    aggs['purchase_amount'] = ['sum']
    aggs['card_id'] = ['count']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    train_test = train_test.join(df_agg)
    print(train_test.shape)
    
    if not(fname.count('lag')):
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / train_test['hist_regist_term']
    elif fname[-4:-1]=='lag':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 1
    elif fname[-2:]=='02':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 3
    elif fname[-2:]=='05':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 6
    elif fname[-2:]=='08':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 9
    elif fname[-2:]=='11':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 12
    
    #========================================================================
    # Save Feature
    elo_save_feature(prefix=prefix, df_feat=train_test)
    #========================================================================    
print('Complete!') 

100%|██████████| 3/3 [00:00<00:00, 58.55it/s]

Make Feature.
(325540, 9)


100%|██████████| 3/3 [00:00<00:00, 53.74it/s]

Make Feature.
(325540, 9)


100%|██████████| 3/3 [00:00<00:00, 57.95it/s]

...

Make Feature.
(325540, 9)


100%|██████████| 3/3 [00:00<00:00, 54.20it/s]

Make Feature.
(325540, 9)


100%|██████████| 12/12 [06:26<00:00, 37.20s/it]

Complete!


In [2]:
prefix = '214_pst'
prefix = '314_pst'
prefix = '414_pst'
import glob
# #========================================================================
# # これは作成済のFeatureを読み込んで計算する
# #========================================================================
combi_list = [ 
    ['new_cat1', 'new_cat0']
    ,['new_cat1_lag1', 'new_cat0_lag1']
    ,['new_cat1_lag2', 'new_cat0_lag2']
    ,['new_cat1', 'new_cat1_lag1'] 
    ,['new_cat1', 'auth1_cat1'] 
    
    ,['new_cat1', 'auth1_cat1_lag0'] 
    ,['new_cat1', 'auth1_cat1_lag02'] 
    ,['new_cat1', 'auth1_cat1_lag05']  
    ,['new_cat1', 'auth1_cat1_lag08']  
    ,['new_cat1', 'auth1_cat1_lag011'] 
    
    ,['new_cat0', 'new_cat0_lag1'] 
    ,['new_cat0', 'auth1_cat0'] 
    ,['new_cat0', 'auth1_cat0_lag0'] 
    ,['new_cat0', 'auth1_cat0_lag02'] 
    ,['new_cat0', 'auth1_cat0_lag05']  
    ,['new_cat0', 'auth1_cat0_lag08']  
    ,['new_cat0', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag0', 'auth1_cat0_lag0'] 
    ,['auth1_cat1_lag02', 'auth1_cat0_lag02'] 
    ,['auth1_cat1_lag05', 'auth1_cat0_lag05'] 
    ,['auth1_cat1_lag011', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag0', 'auth1_cat1_lag02'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag02'] 
    ,['auth1_cat1_lag0', 'auth1_cat1_lag05'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag05'] 
    ,['auth1_cat1_lag0', 'auth1_cat1_lag08'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag08'] 
    ,['auth1_cat1_lag0', 'auth1_cat1_lag011'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag02', 'auth1_cat1_lag05'] 
    ,['auth1_cat0_lag02', 'auth1_cat0_lag05'] 
    ,['auth1_cat1_lag02', 'auth1_cat1_lag08'] 
    ,['auth1_cat0_lag02', 'auth1_cat0_lag08'] 
    ,['auth1_cat1_lag02', 'auth1_cat1_lag011'] 
    ,['auth1_cat0_lag02', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag05', 'auth1_cat1_lag08'] 
    ,['auth1_cat0_lag05', 'auth1_cat0_lag08'] 
    ,['auth1_cat1_lag05', 'auth1_cat1_lag011'] 
    ,['auth1_cat0_lag05', 'auth1_cat0_lag011'] 
]

comp_cols = [
     'CLV'
#     ,'duration'
]

tmp_feature_list = glob.glob(f'../features/1_first_valid/{prefix}*.gz')
                   
feature_list = []
for f in tmp_feature_list:
    if f.count('pst_ratio_') or f.count('pst_diff_'):continue
    for col in comp_cols:
        if f.count(col):
            feature_list.append(f)
            
base = utils.read_df_pkl('../input/base_first*0*')
p_list = utils.parallel_load_data(path_list=feature_list)
df_feat = pd.concat(p_list, axis=1)
train_test = pd.concat([base[key], df_feat], axis=1)

for (fm1, fm2) in tqdm(combi_list):
    for col in comp_cols:
#         203_pst_auth0_lag02_monthly_avg_purchase_amount_min
        try:
            train_test[f"ratio_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] / train_test[f"{prefix}_{fm2}_{col}@"]
            train_test[f"diff_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] - train_test[f"{prefix}_{fm2}_{col}@"]
        except KeyError:
#             print(fm1, fm2, col)
            continue
        
print(train_test.shape)
ratio_diff_cols = [col for col in train_test.columns if col[:5]=='ratio' or col[:4]=='diff']
elo_save_feature(prefix, train_test[ratio_diff_cols], feat_check=False)

100%|██████████| 39/39 [00:00<00:00, 464.64it/s]


(325540, 63)
